In [1]:
from newspaper import Article
import nltk
import newspaper
import pandas as pd

In [2]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import nltk

In [3]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
import string

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction import DictVectorizer

In [23]:
import json
import operator

# Read Data

In [4]:
header = ["GlobalEventId", "sqldate", "MonthYear","Year",
               "Actor1Code","Actor1Name","Actor1CountryCode","Actor1KnownGroupCode","Actor1Type1Code",
               "Actor2Code","Actor2Name","Actor2CountryCode","Actor2KnownGroupCode","Actor2Type1Code",
               "IsRootEvent","EventCode","EventBaseCode","EventRootCode",
               "QuadClass","GoldsteinScale","NumMentions","NumSources","NumArticles","AvgTone",
               "Actor1Geo_FullName","Actor1Geo_CountryCode","Actor1Geo_ADM1Code","Actor1Geo_Lat","Actor1Geo_Long",
               "Actor2Geo_FullName","Actor2Geo_CountryCode","Actor2Geo_ADM1Code","Actor2Geo_Lat","Actor2Geo_Long",
               "ActionGeo_FullName","ActionGeo_CountryCode","ActionGeo_ADM1Code","ActionGeo_Lat","ActionGeo_Long",
               "SOURCEURL"]

In [6]:
#ubah path_clean sesuai tempat clean di lokal
path_clean = '2016-clean.csv'
data_gdelt = pd.read_csv(path_clean, delimiter = ',', encoding = "ISO-8859-1", names=header)
data_gdelt.sample(4)

,GlobalEventId,sqldate,MonthYear,Year,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1Type1Code,Actor2Code,...,Actor2Geo_CountryCode,Actor2Geo_ADM1Code,Actor2Geo_Lat,Actor2Geo_Long,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,SOURCEURL
212279,512891692,20160218,201602,2016,USA,UNITED STATES,USA,NaN,NaN,USALEG,...,US,USSC,33.8191,-80.9066,"South Carolina, United States",US,USSC,33.8191,-80.9066,http://www.willmarradio.com/news/politics/ted-...
1060301,584017398,20160929,201609,2016,BUS,BANK,NaN,NaN,BUS,LAB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://wprl.org/post/wells-fargo-ceo-john-stum...
1126276,590147690,20161018,201610,2016,JUD,SUPREME COURT,NaN,NaN,JUD,IND,...,IN,IN36,29.5500,77.9167,"Sikri, Uttar Pradesh, India",IN,IN36,29.5500,77.9167,http://zeenews.india.com/news/supreme-court-ad...
1355130,610389422,20161221,201612,2016,USA,UNITED STATES,USA,NaN,NaN,JUD,...,US,USMS,32.7673,-89.6812,"Mississippi, United States",US,USMS,32.7673,-89.6812,http://www.mysanantonio.com/news/crime/article...


In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
# contoh 1 URL
sample=data_gdelt['SOURCEURL'][0]
sample
sample_ID = data_gdelt['GlobalEventId'][0]

In [9]:
res = {}

# Fungsi Scrapping

In [10]:
def scrap_article(gid, source):
    article = Article(source)
    article.download()
    article.parse()
    article.nlp()
    
    #jadiin lowercase
    txt = article.text.lower()
    #hapus \n \t dll
    txt_clean = ''.join([x for x in txt if x in string.ascii_letters + '\'- '])
    #jadikan kata perkata
    word_tokens = word_tokenize(txt_clean)
    #filter kata: yang disimpan cuma kata2 yang ga ada di stopword
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    print(type(filtered_sentence))
    #hitung kemunculan tiap kata
    gid_str = str(gid)
    aa = dict((Counter(filtered_sentence)))
    res[gid_str] = aa
    return aa
    

In [11]:
for x in range(8):
    # contoh 1 URL
    try:
        sample_=data_gdelt['SOURCEURL'][x]
        sample_ID_ = data_gdelt['GlobalEventId'][x]
        aaa = scrap_article(sample_ID_, sample_)
        print(aaa, type(aaa))
    except:
        print('gagal', sample_)
        pass

<class 'list'>
{'close': 1, 'rochester': 6, 'man': 2, 'accused': 2, 'isis': 6, 'sympathizer': 2, 'planned': 5, 'kill': 7, 'patrons': 2, 'restaurantbar': 1, 'new': 10, 'year': 7, "'s": 17, 'eve': 8, 'federal': 5, 'authorities': 6, 'allege': 2, 'angie': 1, 'nassargary': 1, 'craigemanuel': 1, 'l': 1, 'lutchman': 20, 'photo': 1, 'providedthe': 1, 'attack': 4, 'say': 4, 'could': 3, 'gruesome': 1, 'assault': 2, 'restaurant': 2, 'bar': 1, 'islamic': 4, 'state': 7, 'armed': 1, 'knives': 2, 'machetei': 1, 'take': 1, 'life': 1, '-year-old': 1, 'emanuel': 3, 'allegedly': 6, 'said': 11, 'tuesday': 1, 'apparent': 1, 'plot': 2, 'merchants': 3, 'grill': 2, 'road': 1, "n't": 3, 'problem': 1, 'thatthe': 1, 'fbi': 12, 'joint': 2, 'terrorism': 3, 'task': 2, 'force': 2, 'wednesday': 2, 'arrested': 3, 'charging': 1, 'offering': 1, 'material': 2, 'support': 2, 'terrorists': 1, 'appeared': 1, 'court': 5, 'thursday': 8, 'morning': 1, 'media': 2, 'alerted': 1, 'advance': 1, 'scheduled': 3, 'return': 1, 'jan': 

<class 'list'>
{'simon': 8, 'danczuk': 34, "'s": 26, 'first': 4, 'wife': 10, 'today': 7, 'threatened': 1, 'change': 2, 'daughter': 4, 'surname': 3, '-year-old': 10, 'thinks': 1, 'mp': 38, 'father': 3, "'dirty": 1, 'old': 6, 'man': 3, "'": 51, 'sexting': 6, 'girlsonia': 1, 'rossington': 5, 'married': 3, 'mr': 18, 'left': 5, "'selfie": 3, 'queen': 3, 'second': 5, 'karen': 11, 'said': 25, "'thoroughly": 2, 'despicable': 3, "man'the": 1, 'campaigned': 1, 'child': 3, 'abuse': 6, 'suspended': 2, 'labour': 18, 'bombarded': 3, 'sophena': 5, 'houlihan': 20, "'horny": 3, 'messages': 22, 'saying': 5, "'you": 2, 'want': 3, 'spank': 4, "you'scroll": 1, 'videoanger': 1, 'sonia': 3, "'appalled": 2, 'behaviour': 1, 'called': 3, "man'an": 1, 'ms': 1, "'is": 1, 'much': 3, 'younger': 2, 'girl': 3, 'wants': 1, 'son': 2, 'george': 1, 'getting': 6, 'hassle': 1, 'university': 1, 'surnamein': 1, 'blow': 1, 'two': 8, 'children': 4, 'revealed': 2, 'divorce': 3, 'came': 4, 'day': 3, 'exposed': 3, 'source': 2, "'

<class 'list'>
{'hidayatullah': 2, 'afridi': 5, 'vows': 1, 'promote': 1, 'pjdcp': 3, 'k-p': 2, 'criticises': 1, 'ppp': 5, 'leadership': 1, 'failurespeshawar': 1, 'senior': 1, 'lawyer': 3, 'former': 3, 'pakistan': 5, 'peoples': 3, 'party': 6, 'leader': 2, 'joined': 1, 'recently-launched': 1, 'justice': 6, 'democratic': 1, 'critic': 1, 'thursday': 1, 'headed': 1, 'chief': 3, 'iftikhar': 2, 'muhammad': 1, 'chaudhryafridi': 1, 'practicing': 1, 'law': 2, 'since': 2, 'previously': 1, 'affiliated': 1, 'lawyers': 1, 'forum': 1, 'said': 4, 'resigned': 1, 'work': 2, 'towards': 1, 'promoting': 1, 'new': 1, 'belongs': 1, 'frontier': 1, 'region': 1, 'fr': 1, 'peshawarspeaking': 1, 'peshawar': 1, 'press': 1, 'club': 1, 'also': 1, 'strongly': 1, 'criticised': 1, 'lack': 1, 'leadershipevery': 1, 'busy': 1, 'hatching': 1, 'conspiracies': 1, 'zardari': 1, 'prevent': 1, 'seizing': 1, 'reins': 1, 'however': 2, 'leaders': 1, 'helplessthe': 1, 'added': 3, 'family': 1, 'loyal': 1, 'last': 1, 'years': 2, 'wou

In [60]:
print(type(aaa))

<class 'dict'>


In [12]:
res

{'498554171': {'close': 1,
  'rochester': 6,
  'man': 2,
  'accused': 2,
  'isis': 6,
  'sympathizer': 2,
  'planned': 5,
  'kill': 7,
  'patrons': 2,
  'restaurantbar': 1,
  'new': 10,
  'year': 7,
  "'s": 17,
  'eve': 8,
  'federal': 5,
  'authorities': 6,
  'allege': 2,
  'angie': 1,
  'nassargary': 1,
  'craigemanuel': 1,
  'l': 1,
  'lutchman': 20,
  'photo': 1,
  'providedthe': 1,
  'attack': 4,
  'say': 4,
  'could': 3,
  'gruesome': 1,
  'assault': 2,
  'restaurant': 2,
  'bar': 1,
  'islamic': 4,
  'state': 7,
  'armed': 1,
  'knives': 2,
  'machetei': 1,
  'take': 1,
  'life': 1,
  '-year-old': 1,
  'emanuel': 3,
  'allegedly': 6,
  'said': 11,
  'tuesday': 1,
  'apparent': 1,
  'plot': 2,
  'merchants': 3,
  'grill': 2,
  'road': 1,
  "n't": 3,
  'problem': 1,
  'thatthe': 1,
  'fbi': 12,
  'joint': 2,
  'terrorism': 3,
  'task': 2,
  'force': 2,
  'wednesday': 2,
  'arrested': 3,
  'charging': 1,
  'offering': 1,
  'material': 2,
  'support': 2,
  'terrorists': 1,
  'appear

In [13]:
#write dictionary to json
with open('data.json', 'a') as fp:
    json.dump(res, fp)

## Transform

In [14]:
#read json file
with open('data.json', 'r') as fp:
    data_counter = json.load(fp)

In [16]:
eventID = []
DictVector = []

In [17]:
#add event ID and bag of words to different array
for k,v in data_counter.items():
    eventID.append(k)
    DictVector.append(v)

In [18]:
# vectorize bag of words from all news
vec = DictVectorizer()
DictArray = vec.fit_transform(DictVector).toarray()

In [19]:
# transform to TFIDF
tfidf_transformer = TfidfTransformer(use_idf=False)
tfidf_transform=tfidf_transformer.transform(DictArray, copy = True)
print(tfidf_transform)
print(DictArray)
print(tfidf_transform.todense())

  (0, 41)	0.32918301113233006
  (0, 66)	0.019363706537195886
  (0, 69)	0.019363706537195886
  (0, 70)	0.019363706537195886
  (0, 71)	0.019363706537195886
  (0, 73)	0.019363706537195886
  (0, 75)	0.03872741307439177
  (0, 76)	0.019363706537195886
  (0, 80)	0.019363706537195886
  (0, 81)	0.019363706537195886
  (0, 82)	0.019363706537195886
  (0, 90)	0.019363706537195886
  (0, 92)	0.05809111961158765
  (0, 95)	0.019363706537195886
  (0, 98)	0.03872741307439177
  (0, 101)	0.019363706537195886
  (0, 104)	0.019363706537195886
  (0, 106)	0.03872741307439177
  (0, 107)	0.1161822392231753
  (0, 108)	0.019363706537195886
  (0, 109)	0.019363706537195886
  (0, 111)	0.019363706537195886
  (0, 112)	0.09681853268597942
  (0, 113)	0.019363706537195886
  (0, 115)	0.03872741307439177
  :	:
  (3, 1072)	0.047727395990334764
  (3, 1076)	0.047727395990334764
  (3, 1103)	0.09545479198066953
  (3, 1113)	0.09545479198066953
  (3, 1134)	0.047727395990334764
  (3, 1152)	0.047727395990334764
  (3, 1157)	0.04772739

In [20]:
#for header
vocabulary = vec.vocabulary_

In [22]:
# sort vocabulary by value (for header)
header = sorted(vocabulary.items(), key=operator.itemgetter(1))
header

[("'", 0),
 ("''", 1),
 ("'a", 2),
 ("'about", 3),
 ("'against", 4),
 ("'almost", 5),
 ("'an", 6),
 ("'appalled", 7),
 ("'are", 8),
 ("'can", 9),
 ("'cheeky", 10),
 ("'communicating", 11),
 ("'d", 12),
 ("'determined", 13),
 ("'dirty", 14),
 ("'discipline", 15),
 ("'disgraceful", 16),
 ("'distracting", 17),
 ("'do", 18),
 ("'flattered", 19),
 ("'get", 20),
 ("'god", 21),
 ("'good", 22),
 ("'greater", 23),
 ("'harassed", 24),
 ("'he", 25),
 ("'hopefully", 26),
 ("'horny", 27),
 ("'i", 28),
 ("'if", 29),
 ("'inappropriate", 30),
 ("'is", 31),
 ("'it", 32),
 ("'let", 33),
 ("'loads", 34),
 ("'look", 35),
 ("'m", 36),
 ("'meet", 37),
 ("'re", 38),
 ("'really", 39),
 ("'right", 40),
 ("'s", 41),
 ("'said", 42),
 ("'selfie", 43),
 ("'sexted", 44),
 ("'sexting", 45),
 ("'she", 46),
 ("'simon", 47),
 ("'spank", 48),
 ("'spanking'she", 49),
 ("'stitched", 50),
 ("'stupid", 51),
 ("'stupid'statement", 52),
 ("'the", 53),
 ("'there", 54),
 ("'thoroughly", 55),
 ("'two", 56),
 ("'very", 57),
 ("'w